<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [42]:
import datetime as dt
import pickle
import sys
from pathlib import Path
from typing import Dict, List

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy
import seaborn as sns
from sqlalchemy.exc import ProgrammingError

In [32]:
print(sys.executable)
print(sys.version)
print(f'SQLalchemy {sqlalchemy.__version__}')

W:\conda-environments\py3\python.exe
3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
SQLalchemy 1.3.21


In [40]:
# Connect to server, create engine

from sqlalchemy import create_engine

SERVER = 'agtst01'
DB_NAME = 'SnippLoyalty_DW_LOEB'

def connect_to_db(server, db_name):
    con_str = f'mssql+pyodbc://@{server}/{db_name}?driver=ODBC Driver 13 for SQL Server'
    engine = create_engine(con_str, fast_executemany=True)
    connection = engine.connect()
    return engine, connection

engine, connection = connect_to_db(SERVER, DB_NAME)

In [72]:
# Load an existing tables_and_views_dict

data_path = Path("../data/2021-05-03_catalyst_validation_data/structure")
file_name = "SnippLoyalty_DW_LOEB_tables_and_views_2021-05-03-12-34-47"

with open(data_path / file_name, "rb") as f:
    tables_views_old = pickle.load(f)

In [73]:
len(tables_views_old["DimMember"])

78

In [123]:
def create_new_empty_cols_dict(
    db_name: str, 
    tables_views_new: Dict[ str, List[str]], 
    connection: sqlalchemy.engine.Connection, 
    n_rows: int = 50
) -> Dict[ str, List[str]]:
    """TODO Use the new dict to check for columns with empty values in
    the first n rows. Print the result.
    """
    print(f"\nStart empty columns check (top {n_rows} rows) for {db_name} ...\n")
    empty_cols_dict = {}

    for table, columns in list(tables_views_new.items())[0:12]:
        try:
            query = f"SELECT TOP {n_rows} * FROM [{table}]"  # ORDER BY NEWID()"
            result = connection.execute(query).fetchall()
            result_df = pd.DataFrame(result, columns=columns)
            result_df.replace("", np.NaN, inplace=True)  # that was a mean one ;-)
            empty_cols = [col for col in result_df.columns if result_df[col].isnull().all() == True]
            print(table, empty_cols)
            if len(empty_cols) > 0:
                empty_cols_dict[table] = empty_cols
        except ProgrammingError:
            print(f"- Table/view '{table}' NOT PARSED! It is not included in analysis.")
            pass

    count_total = len(list(tables_views_new.keys()))
    count_empty = len(list(empty_cols_dict.keys()))
    count_empty_cols = sum([len(v) for v in empty_cols_dict.values()])
    print(f"{count_empty} out of total {count_total} tables have \
          a total of {count_empty_cols} empty columns.\n")

    for key, value in empty_cols_dict.items():
        print(f"- {key}: {value}")
        
    return empty_cols_dict

In [124]:
tst = create_new_empty_cols_dict("SnippLoyalty_DW_LOEB", tables_views_old, connection)


Start empty columns check (top 50 rows) for SnippLoyalty_DW_LOEB ...

_RFM []
DimAccountStatus []
DimAddressValidStatus []
DimAgeRange []
DimAvatar ['LastName', 'SizingCode']
DimBookingAddress ['BookingAddressName', 'BookingAddressSK', 'City', 'ContactNumber', 'CountryAK', 'CountryCode', 'CountryName', 'CreateTimestamp', 'DataSourceId', 'EmailAddress', 'FirstName', 'HouseNumber']
DimBookingAddressCategory []
DimBookingAddressType []
DimClient []
DimCurrency []
DimDate []
DimDeliveryAddress []
2 out of total 87 tables have           a total of 14 empty columns.

- DimAvatar: ['LastName', 'SizingCode']
- DimBookingAddress: ['BookingAddressName', 'BookingAddressSK', 'City', 'ContactNumber', 'CountryAK', 'CountryCode', 'CountryName', 'CreateTimestamp', 'DataSourceId', 'EmailAddress', 'FirstName', 'HouseNumber']


In [112]:
tst

{}

In [14]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.rand(2, 3), columns=list('abc'))
df["d"] = "a"
df = df.select_dtypes(include=np.number)
df

,a,b,c
0,0.346903,0.523191,0.815546
1,0.858489,0.521185,0.999333


In [4]:
df_v = df.to_numpy()
df2_v = df.to_numpy()

In [5]:
df_diff = pd.DataFrame(df_v - df2_v, columns = df.columns)

In [6]:
df_diff

,a,b,c
0,0.0,0.0,0.0
1,0.0,0.0,0.0


In [10]:
for row in df_v:
    for val in row:
        print(val)

0.77493480500492
0.18505757868082395
0.1317477623528206
0.5656333721308152
0.4449672672426237
0.934701660893862


In [20]:
def return_subtraction_df(df_1: pd.DataFrame, df_2: pd.DataFrame) -> pd.DataFrame:
    """TODO Docstring"""
    num_cols = df_1.select_dtypes(include=np.number).columns
    df_1_num_values = df_1.select_dtypes(include=np.number).to_numpy()
    df_2_num_values = df_1.select_dtypes(include=np.number).to_numpy()
    df_diff_values = df_1_num_values - df_2_num_values
    df_diff = pd.DataFrame(df_diff_values, columns=num_cols, index=df_1["year_month"])
    return df_diff

In [17]:
ft = pd.read_parquet("ft")
fti = pd.read_parquet("fti")

In [18]:
ft

,year_month,value,n_trxs,n_members,max_date,date_db_check
0,202011,6310949.25,87761,33891,2020-11-30,2021-05-11
1,202012,4240550.85,68071,30098,2020-12-31,2021-05-11
2,202101,2226356.75,38652,19158,2021-01-30,2021-05-11
3,202102,414803.35,10497,5727,2021-02-27,2021-05-11
4,202103,3945794.75,56928,26403,2021-03-31,2021-05-11
5,202104,3293429.00,46483,23050,2021-04-30,2021-05-11


In [19]:
fti

,year_month,amount,n_trxs,max_date,date_db_check
0,202011,6310949.25,87761,2020-11-30,2021-05-11
1,202012,4240550.85,68071,2020-12-31,2021-05-11
2,202101,2226356.75,38652,2021-01-30,2021-05-11
3,202102,414803.35,10497,2021-02-27,2021-05-11
4,202103,3945815.15,56928,2021-03-31,2021-05-11
5,202104,3293429.00,46483,2021-04-30,2021-05-11


In [21]:
return_subtraction_df(ft, fti)

,n_trxs,n_members
year_month,,
202011,0,0
202012,0,0
202101,0,0
202102,0,0
202103,0,0
202104,0,0


In [43]:
td = {"raph_1": "a", "raph_2": "b", "reb_x": "c", "raph_3": "test"}
# tdraph = {x: y for x, y in td.items() if x.startswith("raph")}
# tdraph

In [44]:
new_keys = []
for k in tdraph.keys():
    try:
        k = k.split("_", 1)[1]
        new_keys.append(k)
    except IndexError:
        new_keys.append(k)


print(new_keys)    
final_dict = dict(zip(new_keys, list(tdraph.values())))
final_dict

['1', '2', 'raph']


{'1': 'a', '2': 'b', 'raph': 'test'}

In [55]:
from typing import Dict, Tuple
import pandas as pd

def grab_and_truncate_df_names_for_vendor(
    vendor_name: str,
    df_dict_new: Dict[str, pd.DataFrame],
    df_dict_old: Dict[str, pd.DataFrame]
) -> Tuple[Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]:
    """Return the df_dicts with the dfs for the relevant vendor only.
    The df_names have the vendor_prefix removed.
    """
    dict_list = []
    for df_dict in [df_dict_new, df_dict_old]:
        df_dict = {k: v for k, v in df_dict.items() if k.startswith(vendor_name)}.copy()
        print(df_dict)
        new_keys = []
        for k in df_dict.keys():
            try:
                k = k.split("_", 1)[1]
                new_keys.append(k)
            except IndexError:
                print(
                    "Every key in the sql_queries dict "
                    "has to start with a vendor name prefix.")
                raise
        print(new_keys)
        print(df_dict)
        if len(new_keys) != list(df_dict.values()):
            raise AssertionError("Problems with vendor name prefixes in "
                                 "the dataframe dictionary keys."
            )

        df_dict = dict(zip(new_keys, list(df_dict.values())))
        dict_list.append(df_dict)

    df_dict_new, df_dict_old = dict_list[0], dict_list[1]
    return df_dict_new, df_dict_old

In [56]:
x, y = grab_and_truncate_df_names_for_vendor( "raph", td, td)

{'raph_1': 'a', 'raph_2': 'b', 'raph_3': 'test'}
['1', '2', '3']
{'raph_1': 'a', 'raph_2': 'b', 'raph_3': 'test'}


AssertionError: Problems with vendor name prefixes in the dataframe dictionary keys.

In [90]:
import numpy as np
df_1 = pd.read_parquet("ft")
df_2 = pd.read_parquet("fti")

df_1
df_2

,year_month,amount,n_trxs,max_date,date_db_check
0,202011,6310949.2500,87761,2020-11-30,2021-05-11
1,202012,4240550.8500,68071,2020-12-31,2021-05-11
2,202101,2226356.7500,38652,2021-01-30,2021-05-11
3,202102,414803.3500,10497,2021-02-27,2021-05-11
4,202103,3945815.1500,56928,2021-03-31,2021-05-11
5,202104,3293429.0000,46483,2021-04-30,2021-05-11


In [130]:
from pandas.api.types import is_numeric_dtype

def return_subtraction_df(
    df_1: pd.DataFrame,
    df_2: pd.DataFrame,
    index_col="yearmon"
) -> pd.DataFrame:
    """Return a dataframe with the values of the numeric cols
    from df_2 subtracted from the numeric cols of df_1. You can pass
    a column that exists in both df as index, default is `yearmon`. 
    Only the values with overlapping index values will be compared.
    Important: Make sure the numeric cols are in the same order in
    both dataframes, else the values will make no sense.
    """

    df_1 = df_1.set_index(index_col).copy()
    df_2 = df_2.set_index(index_col).copy()
    
    overlapping_index_values = sorted(list(set(df_1.index.intersection(df_2.index))))
    print(overlapping_index_values)
    num_cols = [col for col in df_1.columns if is_numeric_dtype(df_1[col])]
    print(num_cols)
    
    print(df_1)
    print(df_2)
    
    df_1_num_values = df_1.loc[overlapping_index_values, num_cols].to_numpy()
    df_2_num_values = df_2.loc[overlapping_index_values, num_cols].to_numpy()
    df_diff_values = df_1_num_values - df_2_num_values
    df_diff = pd.DataFrame(
        df_diff_values,
        columns=num_cols,
        index=sorted(overlapping_index_values)
    )
    return df_diff

In [137]:
for col in df_1:
    print(df_1[col].dtype.kind)

O
O
i
i
O
O


In [131]:
return_subtraction_df(df_1, df_2, "year_month")

['202011', '202012', '202101', '202102', '202103', '202104']
['n_trxs', 'n_members']
                   value  n_trxs  n_members    max_date date_db_check
year_month                                                           
202011      6310949.2500   87761      33891  2020-11-30    2021-05-11
202012      4240550.8500   68071      30098  2020-12-31    2021-05-11
202101      2226356.7500   38652      19158  2021-01-30    2021-05-11
202102       414803.3500   10497       5727  2021-02-27    2021-05-11
202103      3945794.7500   56928      26403  2021-03-31    2021-05-11
202104      3293429.0000   46483      23050  2021-04-30    2021-05-11
                  amount  n_trxs    max_date date_db_check
year_month                                                
202011      6310949.2500   87761  2020-11-30    2021-05-11
202012      4240550.8500   68071  2020-12-31    2021-05-11
202101      2226356.7500   38652  2021-01-30    2021-05-11
202102       414803.3500   10497  2021-02-27    2021-05-11
2

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['n_members'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [111]:
df_1.loc[[1, 2], ['n_trxs', 'n_members']]

,n_trxs,n_members
1,68071,30098
2,38652,19158
